In [6]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

import cv2

import time
from itertools import product
from skimage.io import imread

from joblib import Parallel, delayed

In [7]:
t = time.time()

img2 = cv2.imread('/oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD589_lossless_aligned_cropped/MD589_0197_lossless_aligned_cropped.tif')

print time.time() - t

6.8001139164


In [29]:
print img2.shape

(11744, 13408, 3)


In [28]:
def compute_descriptors(keypoints):
    cv_keypoints = [cv2.KeyPoint(x,y, patch_size) for x, y in keypoints]
    _, descriptors = sift.compute(img, cv_keypoints)
    return descriptors

In [16]:
ys, xs = np.meshgrid(np.arange(half_size, img_h-half_size, stride), 
                     np.arange(half_size, img_w-half_size, stride),
                     indexing='xy')

sample_locations = np.c_[xs.flat, ys.flat]

In [18]:
all_descriptors = np.empty((sample_locations.shape[0], 128), np.int)

In [25]:
img_h, img_w = img2.shape[:2]

t = time.time()

n_partition = 16

# Each iteration, deal with only a fixed-sized (5000 x 5000) part of the image,
# so we can use all 16 cores within each iteration, 
# otherwise the memory pressure is too high, and the performance degrades drastically

for ymin, xmin in product(np.arange(0, img_h, 5000), np.arange(0, img_w, 5000)):
    
    ymax = ymin + 5000
    xmax = xmin + 5000
    
    img = img2[ymin:ymax+1, xmin:xmax+1]

    sift = cv2.SIFT()

    patch_size = 16
    stride = 56
    half_size = patch_size / 2

    h, w = img.shape[:2]

    in_current_segment = \
    (sample_locations[:,1] >= ymin) & \
    (sample_locations[:,1] <= ymax) & \
    (sample_locations[:,0] >= xmin) & \
    (sample_locations[:,0] <= xmax)
    
    partitioned_locations = np.array_split(sample_locations[in_current_segment], n_partition)
    ret = Parallel(n_jobs=n)(delayed(compute_descriptors)(kps) for kps in partitioned_locations)
    all_descriptors[in_current_segment] = np.vstack(ret)
        
print time.time() - t

14.5891628265
